In [12]:
import googlemaps 
import datetime
import pandas as pd
import json
import os

https://github.com/googlemaps/google-maps-services-python/blob/master/googlemaps/distance_matrix.py

In [2]:
API_key = '' 
gmaps = googlemaps.Client(key=API_key)

In [67]:
location_df = pd.read_excel('others/7-11_location.xlsx')
store_list = []
for i in range(len(location_df)):
    row = location_df.loc[i, :]
    store_list.append({
        "name": row['Store Name'], 
        "address": row['Store Address'], 
        "coordinate": (row['latitude'], row['longitude'])
    })

time_set = datetime.datetime(2022, 12, 9, 0, 0, 0, 0)

In [74]:
time_set

datetime.datetime(2022, 12, 9, 0, 0)

In [66]:
#result

{'destination_addresses': ['No. 16, Xining Rd, Wanhua District, Taipei City, Taiwan 108'],
 'origin_addresses': ['No. 111-2, Emei St, Wanhua District, Taipei City, Taiwan 108'],
 'rows': [{'elements': [{'distance': {'text': '0.6 km', 'value': 551},
     'duration': {'text': '4 mins', 'value': 210},
     'duration_in_traffic': {'text': '3 mins', 'value': 168},
     'status': 'OK'}]}],
 'status': 'OK'}

In [68]:
output_file = 'adjacency_matrix.json'

if os.path.exists(output_file):
    print('open file')
    with open(file=output_file, mode='r', encoding='utf-8') as reader:
        adjacency_matrix = json.load(reader)
else:
    print('create file')
    adjacency_matrix = {}
    for i in store_list:
        adjacency_matrix[i['name']] = {}
        for j in store_list:
            if i['name'] == j['name']:
                adjacency_matrix[i['name']][j['name']] = 0
            else:
                adjacency_matrix[i['name']][j['name']] = None
    with open(file=output_file, mode='w', encoding='utf-8') as writer:
        json.dump(obj=adjacency_matrix, fp=writer, ensure_ascii=False, indent=4)

create file


In [69]:
# 用於確認是否通通抓完距離了
def valid_edge():
    cnt = 0
    for src in adjacency_matrix:
        for dest in adjacency_matrix:
            if adjacency_matrix[src][dest] != None:
                cnt += 1
    return cnt


def get_dist(origins, destination):
    result = gmaps.distance_matrix(origins, 
                                   destination, 
                                   mode='driving', 
                                   departure_time=time_set)
    dist = result["rows"][0]["elements"][0]["distance"]["value"]  # 得到公尺數
    return dist/1000

In [70]:
def get_dist_matrix():
    index = 0
    total_ = len(store_list)*len(store_list)
    for origins in store_list:
        for destination in store_list:
            index += 1
            origin_store, dest_store = origins['name'], destination['name']
            # 已經有距離了就跳過
            if adjacency_matrix[origin_store][dest_store] != None:
                continue

            print(f'{index}/{total_}, {origin_store} <-> {dest_store}')
            distance = get_dist(origins['coordinate'], destination['coordinate'])
            adjacency_matrix[origin_store][dest_store] = distance

In [71]:
cnt = valid_edge()
print('cnt = {}'.format(cnt))

while cnt != len(store_list) * len(store_list):
    get_dist_matrix()
    newcnt = valid_edge()
    if cnt == newcnt:
        print('[ERROR] I\'m a robot')
        break
    cnt = newcnt
    print('cnt = {}'.format(cnt))

with open(file=output_file, mode='w', encoding='utf-8') as writer:
    json.dump(obj=adjacency_matrix, fp=writer, ensure_ascii=False, indent=4)

cnt = 31
2/961, 新峨嵋 <-> 開寧
3/961, 新峨嵋 <-> 六福
4/961, 新峨嵋 <-> 鑫杭
5/961, 新峨嵋 <-> 新南
6/961, 新峨嵋 <-> 仁金
7/961, 新峨嵋 <-> 丹陽
8/961, 新峨嵋 <-> 稻江
9/961, 新峨嵋 <-> 榮金
10/961, 新峨嵋 <-> 建龍
11/961, 新峨嵋 <-> 威克
12/961, 新峨嵋 <-> 金蓬
13/961, 新峨嵋 <-> 明美
14/961, 新峨嵋 <-> 明水
15/961, 新峨嵋 <-> 吉安
16/961, 新峨嵋 <-> 永明
17/961, 新峨嵋 <-> 天津
18/961, 新峨嵋 <-> 六條通
19/961, 新峨嵋 <-> 濟南
20/961, 新峨嵋 <-> 黎元
21/961, 新峨嵋 <-> 復昌
22/961, 新峨嵋 <-> 教育大學
23/961, 新峨嵋 <-> 安松
24/961, 新峨嵋 <-> 合維
25/961, 新峨嵋 <-> 中廣
26/961, 新峨嵋 <-> 福中
27/961, 新峨嵋 <-> 嘉馥
28/961, 新峨嵋 <-> 道生
29/961, 新峨嵋 <-> 松聯
30/961, 新峨嵋 <-> 松高
31/961, 新峨嵋 <-> 光復
32/961, 開寧 <-> 新峨嵋
34/961, 開寧 <-> 六福
35/961, 開寧 <-> 鑫杭
36/961, 開寧 <-> 新南
37/961, 開寧 <-> 仁金
38/961, 開寧 <-> 丹陽
39/961, 開寧 <-> 稻江
40/961, 開寧 <-> 榮金
41/961, 開寧 <-> 建龍
42/961, 開寧 <-> 威克
43/961, 開寧 <-> 金蓬
44/961, 開寧 <-> 明美
45/961, 開寧 <-> 明水
46/961, 開寧 <-> 吉安
47/961, 開寧 <-> 永明
48/961, 開寧 <-> 天津
49/961, 開寧 <-> 六條通
50/961, 開寧 <-> 濟南
51/961, 開寧 <-> 黎元
52/961, 開寧 <-> 復昌
53/961, 開寧 <-> 教育大學
54/961, 開寧 <-> 安松
55/961, 開寧 <-> 合維
56/961, 

448/961, 吉安 <-> 明水
450/961, 吉安 <-> 永明
451/961, 吉安 <-> 天津
452/961, 吉安 <-> 六條通
453/961, 吉安 <-> 濟南
454/961, 吉安 <-> 黎元
455/961, 吉安 <-> 復昌
456/961, 吉安 <-> 教育大學
457/961, 吉安 <-> 安松
458/961, 吉安 <-> 合維
459/961, 吉安 <-> 中廣
460/961, 吉安 <-> 福中
461/961, 吉安 <-> 嘉馥
462/961, 吉安 <-> 道生
463/961, 吉安 <-> 松聯
464/961, 吉安 <-> 松高
465/961, 吉安 <-> 光復
466/961, 永明 <-> 新峨嵋
467/961, 永明 <-> 開寧
468/961, 永明 <-> 六福
469/961, 永明 <-> 鑫杭
470/961, 永明 <-> 新南
471/961, 永明 <-> 仁金
472/961, 永明 <-> 丹陽
473/961, 永明 <-> 稻江
474/961, 永明 <-> 榮金
475/961, 永明 <-> 建龍
476/961, 永明 <-> 威克
477/961, 永明 <-> 金蓬
478/961, 永明 <-> 明美
479/961, 永明 <-> 明水
480/961, 永明 <-> 吉安
482/961, 永明 <-> 天津
483/961, 永明 <-> 六條通
484/961, 永明 <-> 濟南
485/961, 永明 <-> 黎元
486/961, 永明 <-> 復昌
487/961, 永明 <-> 教育大學
488/961, 永明 <-> 安松
489/961, 永明 <-> 合維
490/961, 永明 <-> 中廣
491/961, 永明 <-> 福中
492/961, 永明 <-> 嘉馥
493/961, 永明 <-> 道生
494/961, 永明 <-> 松聯
495/961, 永明 <-> 松高
496/961, 永明 <-> 光復
497/961, 天津 <-> 新峨嵋
498/961, 天津 <-> 開寧
499/961, 天津 <-> 六福
500/961, 天津 <-> 鑫杭
501/961, 天津 <-> 新南
502/

887/961, 松聯 <-> 濟南
888/961, 松聯 <-> 黎元
889/961, 松聯 <-> 復昌
890/961, 松聯 <-> 教育大學
891/961, 松聯 <-> 安松
892/961, 松聯 <-> 合維
893/961, 松聯 <-> 中廣
894/961, 松聯 <-> 福中
895/961, 松聯 <-> 嘉馥
896/961, 松聯 <-> 道生
898/961, 松聯 <-> 松高
899/961, 松聯 <-> 光復
900/961, 松高 <-> 新峨嵋
901/961, 松高 <-> 開寧
902/961, 松高 <-> 六福
903/961, 松高 <-> 鑫杭
904/961, 松高 <-> 新南
905/961, 松高 <-> 仁金
906/961, 松高 <-> 丹陽
907/961, 松高 <-> 稻江
908/961, 松高 <-> 榮金
909/961, 松高 <-> 建龍
910/961, 松高 <-> 威克
911/961, 松高 <-> 金蓬
912/961, 松高 <-> 明美
913/961, 松高 <-> 明水
914/961, 松高 <-> 吉安
915/961, 松高 <-> 永明
916/961, 松高 <-> 天津
917/961, 松高 <-> 六條通
918/961, 松高 <-> 濟南
919/961, 松高 <-> 黎元
920/961, 松高 <-> 復昌
921/961, 松高 <-> 教育大學
922/961, 松高 <-> 安松
923/961, 松高 <-> 合維
924/961, 松高 <-> 中廣
925/961, 松高 <-> 福中
926/961, 松高 <-> 嘉馥
927/961, 松高 <-> 道生
928/961, 松高 <-> 松聯
930/961, 松高 <-> 光復
931/961, 光復 <-> 新峨嵋
932/961, 光復 <-> 開寧
933/961, 光復 <-> 六福
934/961, 光復 <-> 鑫杭
935/961, 光復 <-> 新南
936/961, 光復 <-> 仁金
937/961, 光復 <-> 丹陽
938/961, 光復 <-> 稻江
939/961, 光復 <-> 榮金
940/961, 光復 <-> 建龍
941/9

In [72]:
pd.DataFrame(adjacency_matrix).head(3)

,新峨嵋,開寧,六福,鑫杭,新南,仁金,丹陽,稻江,榮金,建龍,...,教育大學,安松,合維,中廣,福中,嘉馥,道生,松聯,松高,光復
新峨嵋,0.000,0.766,0.791,3.167,4.829,3.892,3.491,6.331,8.031,5.897,...,8.747,4.843,6.817,4.502,9.711,9.365,7.201,8.513,7.827,8.639
開寧,0.551,0.000,0.255,2.537,4.199,3.261,2.860,5.424,7.124,5.283,...,7.846,4.337,6.231,3.872,9.391,9.045,6.881,8.194,7.507,8.320
六福,0.292,0.336,0.000,2.981,4.643,3.705,3.305,6.145,7.845,5.711,...,8.560,4.754,6.631,4.316,9.525,9.179,7.015,8.327,7.641,8.453


In [73]:
pd.DataFrame(adjacency_matrix).to_excel("711_distance.xlsx")